In [1]:
# What version of Python do you have?
import sys
import os
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
import re
import mne
import pathlib
import openpyxl
from datetime import datetime
import pytz
import random
import os
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from pathlib import Path
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, Sequential
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, multilabel_confusion_matrix, ConfusionMatrixDisplay, confusion_matrix, plot_confusion_matrix
from PIL import Image
%matplotlib inline

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Init Plugin
Init Graph Optimizer
Init Kernel
Tensor Flow Version: 2.5.0
Keras Version: 2.5.0

Python 3.9.7 (default, Sep 16 2021, 23:53:23) 
[Clang 12.0.0 ]
Pandas 1.3.1
Scikit-Learn 0.24.2
GPU is available


In [9]:
resnet152_input_shape = (224, 224)
inceptionV3_input_shape = (299, 299)
batch_size = 1
print(os.getcwd())
test_df = pd.read_csv("csv/tester.csv")
labels = test_df['class'].unique()

/Users/niklashjort/Desktop/Notes/Speciale/projects/MIT_Model


In [4]:
model = tf.keras.models.load_model("h5_files/ten_epoch_onlytrain_model.h5")

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-10-25 23:01:26.008484: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-10-25 23:01:26.009145: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
model.compile(
    loss="categorical_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

In [6]:
generator = ImageDataGenerator(
  rescale=1./255,
)

In [11]:
inc_test_gen = generator.flow_from_directory("test_images", batch_size=batch_size, target_size=inceptionV3_input_shape, classes=list(labels), shuffle=False)

Found 163 images belonging to 4 classes.


In [14]:
res_test_gen = generator.flow_from_directory("test_images", batch_size=batch_size, target_size=resnet152_input_shape, classes=list(labels), shuffle=False)

Found 163 images belonging to 4 classes.


In [15]:
def generator_test():
     while True:
        X1i = inc_test_gen.next()
        X2i = res_test_gen.next()
        yield [X1i[0], X2i[0]], X1i[1]

In [16]:
y_true = inc_test_gen.classes
y_true

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [17]:
number_of_steps = inc_test_gen.samples // batch_size
number_of_steps

163

In [18]:
y_pred = model.predict(generator_test(), number_of_steps)

2021-10-25 23:21:11.340692: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-10-25 23:21:11.343320: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-10-25 23:21:12.210419: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


In [ ]:
y_pred_max = y_pred.armax(axis=-1)

In [ ]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix, ConfusionMatrixDisplay, confusion_matrix, plot_confusion_matrix

In [ ]:
clf_report = classification_report(y_true=y_true, y_pred=y_pred_max, target_names=labels)

In [ ]:
print(clf_report)

In [ ]:
confusion_matrix = confusion_matrix(y_true=y_true, y_pred=list(y_pred_max))

confusion_matrix

In [ ]:
import matplotlib

In [ ]:
ConfusionMatrixDisplay(confusion_matrix=confusion_matrix, display_labels=labels_as_array).plot(cmap='Blues')
plt.title("10 Epoch : chb01=train : chb01valida=test")
fig = matplotlib.pyplot.gcf()
fig.set_size_inches(18.5, 10.5)
plt.savefig("hi2.png")